# 1. Importações

In [ ]:
import time
from xgboost import XGBClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 2. Leitura dos dados

In [ ]:
path = '/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/01. Dados/01. Dados para treino e teste/'
x_train = pd.read_csv(path + 'x_train.csv', index_col=False)
x_test = pd.read_csv(path + 'x_test.csv', index_col=False)
y_train = pd.read_csv(path + 'y_train.csv', index_col=False, usecols=['classification'])
y_test = pd.read_csv(path + 'y_test.csv', index_col=False, usecols=['classification'])

len(x_train), len(x_test), len(y_train), len(y_test)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1106, 301), (277, 301), (1106, 1), (277, 1))

# 3. Definição da classe do classificador XGBoost

In [ ]:
class XGBoosterClassifier():
  def __init__(self,random_state=300):
    self.model = XGBClassifier(tree_method = "gpu_hist", random_state=random_state)

  def fit(X,y):
    self.model.fit(X,y)

  def predict(y):
    self.model.predict(y)


# 4. Exemplo de uso

In [ ]:
param_grid = {
    "learning_rate": [0.1, 0.01],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "subsample": [0.6, 0.8, 1.0],
    "max_depth": [2, 3, 4],
    "n_estimators": [100, 200, 300, 400],
    "reg_lambda": [1, 1.5, 2],
    "gamma": [0, 0.1, 0.3],
}

seed = 300
classifier = XGBoosterClassifier(seed)

num_folds = 10
kfold = StratifiedKFold(n_splits=num_folds)

# create the grid search object
n_iter=50
grid = RandomizedSearchCV(
    estimator=classifier, 
    param_distributions=param_grid,
    cv=kfold,
    n_jobs=-1,
    n_iter=n_iter,
    refit="AUC",
)


In [ ]:
# create a default XGBoost classifier
param_grid = {
    "learning_rate": [0.1, 0.01],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "subsample": [0.6, 0.8, 1.0],
    "max_depth": [2, 3, 4],
    "n_estimators": [100, 200, 300, 400],
    "reg_lambda": [1, 1.5, 2],
    "gamma": [0, 0.1, 0.3],
}

# define the eval set and metric
eval_set = [(x_train, y_train), (x_test, y_test)]
eval_metric = ["auc","error"]

model = XGBClassifier(n_estimators=500, random_state=300)
%time best_model = model.fit(x_train, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=False)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: user 9.19 s, sys: 22.1 ms, total: 9.22 s
Wall time: 9.22 s


In [ ]:
# final model assessment
pred_test = model.predict(x_test)
pred_train = model.predict(x_train)
print('Train Accuracy: ', accuracy_score(y_train, pred_train))
print('Test Accuraccy: ', accuracy_score(y_test, pred_test))
print('Classification Report:')

Train Accuracy:  1.0
Test Accuraccy:  0.851985559566787
Classification Report:


In [ ]:
print(classification_report(y_test,pred_test))

              precision    recall  f1-score   support

           0       0.83      0.86      0.85       130
           1       0.87      0.84      0.86       147

    accuracy                           0.85       277
   macro avg       0.85      0.85      0.85       277
weighted avg       0.85      0.85      0.85       277



Com randomsearch


In [ ]:
# create a default XGBoost classifier
model = XGBClassifier(
    tree_method = "gpu_hist", 
    random_state=seed, 
    eval_metric=["error", "auc"]
)

In [ ]:
# Create the grid search parameter grid and scoring funcitons
param_grid = {
    "learning_rate": [0.1, 0.01],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "subsample": [0.6, 0.8, 1.0],
    "max_depth": [2, 3, 4],
    "n_estimators": [100, 200, 300, 400],
    "reg_lambda": [1, 1.5, 2],
    "gamma": [0, 0.1, 0.3],
}
scoring = {
    'AUC': 'roc_auc', 
    'Accuracy': make_scorer(accuracy_score)
}

In [ ]:
num_folds = 10
kfold = StratifiedKFold(n_splits=num_folds)

# create the grid search object
n_iter=50
grid = RandomizedSearchCV(
    estimator=model, 
    param_distributions=param_grid,
    scoring=scoring,
    cv=kfold,
    n_jobs=-1,
    n_iter=n_iter,
    refit="AUC",
)


In [ ]:
# fit grid search
%time best_model = grid.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: user 2.65 s, sys: 376 ms, total: 3.02 s
Wall time: 6min 21s


In [ ]:
print(f'Best score: {best_model.best_score_}')
print(f'Best model: {best_model.best_params_}')

Best score: 0.915194711941907
Best model: {'subsample': 0.8, 'reg_lambda': 2, 'n_estimators': 300, 'max_depth': 2, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 0.6}


In [ ]:
pred_test = best_model.predict(x_test)
pred_train = best_model.predict(x_train)
print('Train Accuracy: ', accuracy_score(y_train, pred_train))
print('Test Accuraccy: ', accuracy_score(y_test, pred_test))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test,pred_test))
print('\nClassification Report:')
print(classification_report(y_test,pred_test))

Train Accuracy:  0.9954792043399638
Test Accuraccy:  0.8592057761732852

Confusion Matrix:
[[113  17]
 [ 22 125]]

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.87      0.85       130
           1       0.88      0.85      0.87       147

    accuracy                           0.86       277
   macro avg       0.86      0.86      0.86       277
weighted avg       0.86      0.86      0.86       277



In [ ]:
pred_test

NameError: ignored